In [1]:
%pip install transformers datasets torch scikit-learn accelerate

Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('LightTai/personalized-email')

/Users/caoyun/miniconda3/envs/ml-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(dataset)
print(dataset['train'][:5])
df = dataset['train'].to_pandas()
print(df.head())
print(df.shape)

DatasetDict({
    train: Dataset({
        features: ['product', 'gender', 'profession', 'hobby', 'email'],
        num_rows: 30
    })
})
{'product': ['piano lessons', 'guitar lessons', 'vacation plans', 'vacation plans', 'vacation plans'], 'gender': ['male', 'male', 'male', 'female', 'female'], 'profession': ['college students', 'college students', 'college students', 'college students', 'company employees'], 'hobby': ['like to play piano', 'like to play piano', 'like swimming', 'like to look at the scenery', 'like to look at the scenery'], 'email': ["Subject: Elevate Your Piano Skills - Exclusive Offer Inside!\n\nHey [Name],\n\nLooking to unlock your piano potential? As a fellow male college student and a passionate piano player, I understand your love for music. That's why I'm thrilled to offer you exclusive piano lessons designed to fit your busy student schedule.\n\nMaster your favorite melodies, refine techniques, and gain a deeper understanding of music theory-all while enjoyin

In [8]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset

model_checkpoint = "postbot/distilgpt2-emailgen"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_function(examples):
    features = [f"{prod} {gen} {prof} {hob}" 
                for prod, gen, prof, hob in zip(examples["product"], 
                                                examples["gender"], 
                                                examples["profession"], 
                                                examples["hobby"])]
    tokenized_inputs = tokenizer(features, truncation=True, padding="max_length", max_length=512)

    # Tokenize the email column which is our target
    tokenized_targets = tokenizer(examples["email"], truncation=True, padding="max_length", max_length=512)

    tokenized_inputs['labels'] = tokenized_targets['input_ids']  # Assign target token ids as labels for training
    return tokenized_inputs


# Assuming 'df' is your DataFrame and it's already loaded
train_df, test_df = train_test_split(df, test_size=0.2)

# Convert the DataFrames back to Hugging Face dataset format if necessary
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["product", "gender", "profession", "hobby", "email"])
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["product", "gender", "profession", "hobby", "email"])
# Assuming 'dataset' is a Hugging Face 'datasets' object
# tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["product", "gender", "profession", "hobby", "email"])


Map (num_proc=4): 100%|██████████| 6/6 [00:00<00:00, 16.26 examples/s]


In [9]:
print(tokenized_train_dataset)

Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 24
})


In [11]:
tokenized_train_dataset[1]

{'__index_level_0__': 14,
 'input_ids': [25119,
  12437,
  4048,
  3710,
  588,
  284,
  467,
  9735,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256

In [12]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

training_args = TrainingArguments(
    output_dir="./model_output",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/Users/caoyun/miniconda3/envs/ml-env/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(
  0%|          | 0/18 [00:00<?, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                              
 33%|███▎      | 6/18 [02:34<04:30, 22.54s/it]

{'eval_loss': 6.531621932983398, 'eval_runtime': 6.6834, 'eval_samples_per_second': 0.898, 'eval_steps_per_second': 0.299, 'epoch': 1.0}


                                               
 67%|██████▋   | 12/18 [04:36<02:09, 21.58s/it]

{'eval_loss': 5.367166042327881, 'eval_runtime': 5.3518, 'eval_samples_per_second': 1.121, 'eval_steps_per_second': 0.374, 'epoch': 2.0}


                                               
100%|██████████| 18/18 [06:02<00:00, 20.16s/it]

{'eval_loss': 4.947717189788818, 'eval_runtime': 5.9565, 'eval_samples_per_second': 1.007, 'eval_steps_per_second': 0.336, 'epoch': 3.0}
{'train_runtime': 362.7972, 'train_samples_per_second': 0.198, 'train_steps_per_second': 0.05, 'train_loss': 6.530542585584852, 'epoch': 3.0}


TrainOutput(global_step=18, training_loss=6.530542585584852, metrics={'train_runtime': 362.7972, 'train_samples_per_second': 0.198, 'train_steps_per_second': 0.05, 'train_loss': 6.530542585584852, 'epoch': 3.0})

In [15]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
print(eval_results)

100%|██████████| 2/2 [00:02<00:00,  1.43s/it]

Perplexity: 140.85
{'eval_loss': 4.947717189788818, 'eval_runtime': 7.3058, 'eval_samples_per_second': 0.821, 'eval_steps_per_second': 0.274, 'epoch': 3.0}
